### Ejercicio Práctico

Asignatura: Data Science y Big Data
Máster en Inteligencia Artificial
Prof. Ronal Muresano


La práctica está compuesta por un conjunto de operaciones en Spark y al final de la práctica debe generar un fichero de tipo notebook que se tendrá que subir a la plataforma. 

La empresa de desarrollo de software CRAN (R studio) quiere que le diseñe una ETL en Spark que permita enriquecer su sistema de logs. Dentro de este proceso se incluye un primer CSV que esta compuesto de los logs y tiene nombre de “dataset.csv” y un segundo fichero que incluye la geografía “geography.csv”. 

Dentro de los campos del fichero de logs tenemos: 


Ver documento

En el fichero de geografía tenemos: 

Ver documento

1)	Lea el fichero y cargue cada dataframe con un esquema predeterminado. Además, rellene los elementos vacíos con 0 para las variables numéricas y “NOT-APPLICABLE” para el resto de string. 

In [0]:
import pyspark.sql.functions as f 
from pyspark.sql.functions import col, sum
from pyspark.sql.types import IntegerType, DoubleType, StringType, BooleanType, ArrayType, NumericType, StructType, StructField


In [0]:
file_path_geo = "dbfs:/FileStore/geography.csv"
file_path_dev = "dbfs:/FileStore/device.csv"

df_geo = spark.read.csv(file_path_geo, header=True, inferSchema=True)
df_dev = spark.read.csv(file_path_dev, header=True, inferSchema=True)
df_geo.show(20)

root
 |-- country_code: string (nullable = true)
 |-- country_name: string (nullable = true)
 |-- subcontinent_name: string (nullable = true)
 |-- continent_name: string (nullable = true)

+------------+--------------------+-----------------+--------------+
|country_code|        country_name|subcontinent_name|continent_name|
+------------+--------------------+-----------------+--------------+
|          DZ|             Algeria|     North Africa|        Africa|
|          MA|             Morocco|     North Africa|        Africa|
|          SD|               Sudan|     North Africa|        Africa|
|          SS|         South Sudan|     North Africa|        Africa|
|          TN|             Tunisia|     North Africa|        Africa|
|          AO|              Angola|Subsaharan Africa|        Africa|
|          BF|        Burkina Faso|Subsaharan Africa|        Africa|
|          BI|             Burundi|Subsaharan Africa|        Africa|
|          BJ|               Benin|Subsaharan Africa

Esta vez use inferSchema ya que en medical_services use un schema creado

In [0]:
df_dev.show(20)

+----------+-------------------+-------+------------+------+------------+----------+--------+-------+-----+
|      date|               time|   size|version_code|  arch|          os|   package| version|country|ip_id|
+----------+-------------------+-------+------------+------+------------+----------+--------+-------+-----+
|2015-12-12|2024-12-29 13:42:10| 257886|       3.2.2|  i386|     mingw32|  HistData|   0.7-6|     CZ|    1|
|2015-12-12|2024-12-29 13:24:37|1236751|       3.2.2|x86_64|     mingw32|   RJSONIO|   1.3-0|     DE|    2|
|2015-12-12|2024-12-29 13:42:35|2077876|       3.2.2|  i386|     mingw32|    UsingR|   2.0-5|     CZ|    1|
|2015-12-12|2024-12-29 13:42:01| 266724|       3.2.2|  i386|     mingw32| gridExtra|   2.0.0|     CZ|    1|
|2015-12-12|2024-12-29 13:00:21|3687766|          NA|    NA|          NA|      lme4|  1.1-10|     DE|    3|
|2015-12-12|2024-12-29 13:08:56|  57429|          NA|    NA|          NA|  testthat|  0.11.0|     DE|    3|
|2015-12-12|2024-12-29 13:08

In [0]:
df_geo.printSchema()
df_dev.printSchema()

root
 |-- country_code: string (nullable = true)
 |-- country_name: string (nullable = true)
 |-- subcontinent_name: string (nullable = true)
 |-- continent_name: string (nullable = true)

root
 |-- date: date (nullable = true)
 |-- time: timestamp (nullable = true)
 |-- size: integer (nullable = true)
 |-- version_code: string (nullable = true)
 |-- arch: string (nullable = true)
 |-- os: string (nullable = true)
 |-- package: string (nullable = true)
 |-- version: string (nullable = true)
 |-- country: string (nullable = true)
 |-- ip_id: integer (nullable = true)



timestamp no lo considero numeric por que una hora 0 podria ser confuso
Uso lo del ejercicio de medical services y lo convierto en una UDF

In [0]:
def fillna_cust(df):
    dataFrame_cols = df.dtypes
    values = {}

    for col, col_type in dataFrame_cols:
        if col_type in ['int', 'double']:
            values[col] = 0
        else:
            values[col] = 'NOT-APPLICABLE'

    return df.fillna(values)

In [0]:
df_geo_new = fillna_cust(df_geo)
df_geo_new.show()

+------------+--------------------+-----------------+--------------+
|country_code|        country_name|subcontinent_name|continent_name|
+------------+--------------------+-----------------+--------------+
|          DZ|             Algeria|     North Africa|        Africa|
|          MA|             Morocco|     North Africa|        Africa|
|          SD|               Sudan|     North Africa|        Africa|
|          SS|         South Sudan|     North Africa|        Africa|
|          TN|             Tunisia|     North Africa|        Africa|
|          AO|              Angola|Subsaharan Africa|        Africa|
|          BF|        Burkina Faso|Subsaharan Africa|        Africa|
|          BI|             Burundi|Subsaharan Africa|        Africa|
|          BJ|               Benin|Subsaharan Africa|        Africa|
|          BW|            Botswana|Subsaharan Africa|        Africa|
|          CD|Congo, Democratic...|Subsaharan Africa|        Africa|
|          CF|Central African R...

In [0]:
df_dev_new = fillna_cust(df_dev)
df_dev_new.show(20)

+----------+-------------------+-------+------------+------+------------+----------+--------+-------+-----+
|      date|               time|   size|version_code|  arch|          os|   package| version|country|ip_id|
+----------+-------------------+-------+------------+------+------------+----------+--------+-------+-----+
|2015-12-12|2024-12-29 13:42:10| 257886|       3.2.2|  i386|     mingw32|  HistData|   0.7-6|     CZ|    1|
|2015-12-12|2024-12-29 13:24:37|1236751|       3.2.2|x86_64|     mingw32|   RJSONIO|   1.3-0|     DE|    2|
|2015-12-12|2024-12-29 13:42:35|2077876|       3.2.2|  i386|     mingw32|    UsingR|   2.0-5|     CZ|    1|
|2015-12-12|2024-12-29 13:42:01| 266724|       3.2.2|  i386|     mingw32| gridExtra|   2.0.0|     CZ|    1|
|2015-12-12|2024-12-29 13:00:21|3687766|          NA|    NA|          NA|      lme4|  1.1-10|     DE|    3|
|2015-12-12|2024-12-29 13:08:56|  57429|          NA|    NA|          NA|  testthat|  0.11.0|     DE|    3|
|2015-12-12|2024-12-29 13:08

2)	Determine el número de paquetes por cada tipo descargado y añádalo como una nueva columna llamada “Num_Package_downloaded” al datraframe original 

In [0]:
package_counts = df_dev_new.groupBy("package").agg(f.count("package").alias("Num_Package_downloaded"))
package_counts.show(20)

df_dev_pkd = df_dev_new.join(package_counts, on="package", how="left")

df_dev_pkd.show(30)


+------------------+----------------------+
|           package|Num_Package_downloaded|
+------------------+----------------------+
|           TH.data|                   532|
|             sharx|                    20|
|           spssDDI|                     8|
|        xpose4data|                     5|
|        websockets|                     3|
|                MM|                    17|
|             rtson|                    12|
|         ade4TkGUI|                    48|
|       BayesVarSel|                    20|
|           bdscale|                    18|
|              JADE|                    28|
|            ncvreg|                    30|
|          hapassoc|                    32|
|           MIIVsem|                    17|
|           waffect|                    17|
|RcmdrPlugin.orloca|                    30|
|           cshapes|                    30|
|            teigen|                    25|
|      RcppParallel|                    28|
|         RcppShark|            

3)	Separe todos los dispositivos cuya arquitectura sea del tipo “x86-64” y realice las siguientes operaciones: 

a.	Concatene la dirección IP, la versión del sistema operativo y la versión de R, este proceso lo debe hacer con una función UDF. 

b.	La columna resultante, inclúyala en el dataframe original como “Device_code”, recuerde hacer un identificador monolítico para poder hacer el mergeo de datasets.

Nota: use un join para hacer este apartado.


In [0]:
def cal_concat(df, col_uid, col_name1, col_name2, col_name3, col_concat_name):
    df_concat = df.withColumn(col_concat_name, f.concat(col(col_name1), col(col_name2), col(col_name3)))
    return df_concat.select(col_concat_name, col_uid)

In [0]:
df_dev_uid = df_dev_pkd.withColumn('uid', f.monotonically_increasing_id())

df_dev_dvc = cal_concat(df_dev_uid, 'uid', 'ip_id', 'os', 'version', 'Device_code')

df_dev_joined = df_dev_uid.join(df_dev_dvc, "uid", "inner")

df_dev_joined.show(20)

+---+----------+----------+-------------------+-------+------------+------+------------+--------+-------+-----+----------------------+--------------------+
|uid|   package|      date|               time|   size|version_code|  arch|          os| version|country|ip_id|Num_Package_downloaded|         Device_code|
+---+----------+----------+-------------------+-------+------------+------+------------+--------+-------+-----+----------------------+--------------------+
|  0|  HistData|2015-12-12|2024-12-29 13:42:10| 257886|       3.2.2|  i386|     mingw32|   0.7-6|     CZ|    1|                   159|       1mingw320.7-6|
|  1|   RJSONIO|2015-12-12|2024-12-29 13:24:37|1236751|       3.2.2|x86_64|     mingw32|   1.3-0|     DE|    2|                  1358|       2mingw321.3-0|
|  2|    UsingR|2015-12-12|2024-12-29 13:42:35|2077876|       3.2.2|  i386|     mingw32|   2.0-5|     CZ|    1|                   151|       1mingw322.0-5|
|  3| gridExtra|2015-12-12|2024-12-29 13:42:01| 266724|       3.

4)	Carge el dataframe de la geografía y haga un join para enriquecer todas las columnas de la geografía en el dataframe original. 

No entendi con que hay que enriquecer, todos los ejercicios fueron sobre devices

5)	Muestre un top 10 de los paquetes más descargados por país, sistema operativo y versión de R 

In [0]:
df_dv_top10 = df_dev_joined.groupBy('country', 'os', 'version', 'package').agg(f.count("package").alias("package_dwn")).orderBy(col("package_dwn").desc()).limit(10)


df_dv_top10.show()

+-------+-------+--------+---------+-----------+
|country|     os| version|  package|package_dwn|
+-------+-------+--------+---------+-----------+
|     US|mingw32|  0.12.2|     Rcpp|       1291|
|     US|mingw32|   1.4.3|  foreach|       1140|
|     US|mingw32|   1.8.3|     plyr|       1128|
|     US|mingw32|   0.9-7|    rJava|       1122|
|     US|mingw32|   1.0.1|  ggplot2|       1024|
|     US|mingw32|   1.0-1|  stringi|       1020|
|     US|mingw32|   1.0.8|iterators|       1016|
|     US|mingw32|1.95-4.7|    RCurl|       1016|
|     US|mingw32|   1.0.0|  stringr|        968|
|     US|mingw32|     1.5| magrittr|        967|
+-------+-------+--------+---------+-----------+



6)	Crear una columna año, mes y día a partir de la columna date

In [0]:
df_dev_date = df_dev_joined.withColumn("year", f.year(df_dev_joined["date"])) \
                 .withColumn("month", f.month(df_dev_joined["date"])) \
                 .withColumn("day", f.dayofmonth(df_dev_joined["date"]))

df_dev_date.show()

+---+----------+----------+-------------------+-------+------------+------+------------+--------+-------+-----+----------------------+--------------------+----+-----+---+
|uid|   package|      date|               time|   size|version_code|  arch|          os| version|country|ip_id|Num_Package_downloaded|         Device_code|year|month|day|
+---+----------+----------+-------------------+-------+------------+------+------------+--------+-------+-----+----------------------+--------------------+----+-----+---+
|  0|  HistData|2015-12-12|2024-12-29 13:42:10| 257886|       3.2.2|  i386|     mingw32|   0.7-6|     CZ|    1|                   159|       1mingw320.7-6|2015|   12| 12|
|  1|   RJSONIO|2015-12-12|2024-12-29 13:24:37|1236751|       3.2.2|x86_64|     mingw32|   1.3-0|     DE|    2|                  1358|       2mingw321.3-0|2015|   12| 12|
|  2|    UsingR|2015-12-12|2024-12-29 13:42:35|2077876|       3.2.2|  i386|     mingw32|   2.0-5|     CZ|    1|                   151|       1min

7)	Guarde el fichero 

In [0]:
df_dev_date.write.mode('overwrite').parquet('def_dev_date')